In [1]:
import pandas as pd
import numpy as np
import get_prices as hist
import tensorflow as tf
from preprocessing import DataProcessing
import pandas_datareader.data as pdr
import fix_yahoo_finance as fix
fix.pdr_override()

In [2]:
start = "2003-01-01"
end = "2018-01-01"

hist.get_stock_data("AAPL", start_date=start, end_date=end)
process = DataProcessing("stock_prices.csv", 0.9)
process.gen_test(10)
process.gen_train(10)

[*********************100%***********************]  1 of 1 downloaded


/Users/oscar/NLP/stockprice_prediction-master/get_prices.py:32: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  stock_data.to_csv("stock_prices.csv")


In [3]:
X_train = process.X_train.reshape((3379, 10, 1)) / 200
Y_train = process.Y_train / 200

X_test = process.X_test.reshape(359, 10, 1) / 200
Y_test = process.Y_test / 200

In [9]:
X_train

array([[[0.0035 ],
        [0.00355],
        [0.00355],
        ...,
        [0.0035 ],
        [0.0035 ],
        [0.00345]],

       [[0.00355],
        [0.00355],
        [0.00355],
        ...,
        [0.0035 ],
        [0.00345],
        [0.0035 ]],

       [[0.00355],
        [0.00355],
        [0.00345],
        ...,
        [0.00345],
        [0.0035 ],
        [0.00335]],

       ...,

       [[0.47035],
        [0.4668 ],
        [0.46775],
        ...,
        [0.4721 ],
        [0.465  ],
        [0.46555]],

       [[0.4668 ],
        [0.46775],
        [0.47115],
        ...,
        [0.465  ],
        [0.46555],
        [0.464  ]],

       [[0.46775],
        [0.47115],
        [0.47305],
        ...,
        [0.46555],
        [0.464  ],
        [0.466  ]]])

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(20, input_shape=(10, 1), return_sequences=True))
model.add(tf.keras.layers.LSTM(20))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.relu))

model.compile(optimizer="adam", loss="mean_squared_error")

model.summary()

model.fit(X_train, Y_train, epochs=200)

print(model.evaluate(X_test, Y_test))

data = pdr.get_data_yahoo("AAPL", "2017-12-19", "2018-01-03")
stock = data["Adj Close"]
X_predict = np.array(stock).reshape((1, 10, 1)) / 200

print(model.predict(X_predict)*200)



In [4]:
#If instead of a full backtest, you just want to see how accurate the model is for a particular prediction, run this:
data = pdr.get_data_yahoo("AAPL", "2017-12-19", "2018-01-03")
stock = data["Adj Close"]

[*********************100%***********************]  1 of 1 downloaded


In [5]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-12-18,174.88,177.20,174.86,176.42,173.07,29421100
2017-12-19,175.03,175.39,174.09,174.54,171.22,27436400
2017-12-20,174.87,175.42,173.25,174.35,171.04,23475600
2017-12-21,174.17,176.02,174.10,175.01,171.69,20949900
2017-12-22,174.68,175.42,174.50,175.01,171.69,16114600
2017-12-26,170.80,171.47,169.68,170.57,167.33,33185500
2017-12-27,170.10,170.78,169.71,170.60,167.36,21498200
2017-12-28,171.00,171.85,170.48,171.08,167.83,16480200
2017-12-29,170.52,170.59,169.22,169.23,166.01,25884400


In [ ]:
X_predict = np.array(stock).reshape((1, 10)) / 200
print(model.predict(X_predict)*200)